## Database Connection

Connecting to the database:

In [1]:
from data import DataManager

imdb_data = DataManager(
    dbname='imdb',
    user='samaher',
    password="CodingIsFun++",
    host='localhost',
    port='5432'
)

Creating 5 dataframes for the tables I extracted from the database: 
- movie
- genre
- prod
- rating
- crew

In [2]:
# Get data from the movie table
movie_data = imdb_data.get_movie()

# Get data from the genre table
genre_data = imdb_data.get_genre()

# Get data from the produced table
prod_data = imdb_data.get_prod()

# Get data from the rating table
rating_data = imdb_data.get_rating()

# Get data from the crew table
crew_data = imdb_data.get_crew()


### Quiz info 

- questions: list of questions from the table movie
- correct answer: the cell in the corresponding row of the column 'year' in the movie table
- options: 3 random options from random cells in the column 'year' in the movie table BUT different from the correct answer
- level: choose the level of difficulty

#### Score

The score is calculated this way: 
wrong answer: 0 points
correct answer:
- 1 if EASY
- 2 if MEDIUM
- 3 if HARD

#### Difficulty Level

- easy: if the movie was released starting from 2010 and has more than 500000 votes
- medium: if the movie was released between 1990 and 2010 and has between 100000 abd 500000
- hard: if the movie was released before 1990 and has less than 100000 votes

In [32]:
def determine_difficulty_level(year, votes):
    if (year >= 2010 and votes >= 500000) or (year < 2010 and votes >= 1000000):
        return 'easy'
    elif 1990 <= year < 2010 and 100000 <= votes < 500000:
        return 'medium'
    elif year < 1990 and votes < 100000:
        return 'hard'
    else:
        return 'unknown'


In [37]:
import random

def generate_movie_question(row, desired_difficulty):
    official_title = row['official_title']
    movie_correct_answer = row['year']

    # Determining difficulty level based on release year and votes
    year = int(row['year'])
    votes = int(row['votes'])
    difficulty_level = determine_difficulty_level(year, votes)

    # Check if the difficulty level matches the desired difficulty
    if difficulty_level == desired_difficulty:
        # Getting all unique years excluding the correct answer
        all_years = list(set(movie_data['year'].unique()))
        all_years.remove(movie_correct_answer)  # Removing the correct answer

        # Selecting 3 more random incorrect options
        movie_options = [movie_correct_answer] + random.sample(all_years, 3)

        # Shuffling the options to randomize their order
        random.shuffle(movie_options)

        # Mapping options to letters (A, B, C, D)
        options_mapping = {chr(ord('A') + i): option for i, option in enumerate(movie_options)}

        # Constructing the question dictionary
        question_dict = {
            'question': f'When was the movie {official_title} released?',
            'options': options_mapping,
            'correct_answer': chr(ord('A') + movie_options.index(movie_correct_answer)),
            'difficulty_level': difficulty_level
        }

        return question_dict
    else:
        return None  # Return None for questions with undesired difficulty

# Update quiz_game to remove the difficulty_level argument
def quiz_game():
    difficulty_levels = ['easy', 'medium', 'hard']

    # Getting user input for difficulty level
    user_difficulty = input("Choosing a difficulty level (easy, medium, hard): ").lower()

    # Validating user input
    while user_difficulty not in difficulty_levels:
        print("Invalid difficulty level. Please choose from: easy, medium, hard")
        user_difficulty = input("Choosing a difficulty level (easy, medium, hard): ").lower()

    # Generating a random row index
    while True:
        row_index = random.randint(0, len(movie_data) - 1)
        row = movie_data.iloc[row_index]

        # Generating the question with the desired difficulty level
        question_info = generate_movie_question(row, user_difficulty)

        if question_info is not None:  # Check if a valid question is generated
            break

    # Printing the question information
    print(question_info['question'])

    # Printing options with letters (A, B, C, D)
    for letter, option in question_info['options'].items():
        print(f"{letter}. {option}")

    # Getting user's choice
    user_choice = input("Entering your choice (A, B, C, D): ").upper()

    # Checking if the user's choice is correct
    is_correct = user_choice == question_info['correct_answer']

    # Calculating and displaying the score
    score = calculate_score(question_info['difficulty_level'], is_correct)

    # Providing feedback on the answer
    if is_correct:
        print("Correct!")
    else:
        print(f"Wrong! The correct answer is: {question_info['correct_answer']}")

    print(f"Your score: {score}")

# Calling the quiz_game function to play the game
quiz_game()


When was the movie Hostel released?
A. 1951
B. 2005
C. 1930
D. 1952
Correct!
Your score: 2


In [22]:
check_row = rating_data[rating_data['official_title'] == 'Elstree Calling']
check_row

,official_title,year,votes,score,scale
53,Elstree Calling,1930,254,5.8,10


## Closing the connection:

In [ ]:
imdb_data.close_connection()